In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import sys
import asyncio

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__


## Local MCP server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": "python",
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain.agents import create_agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model="meta/llama-3.3-70b-instruct")

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='f3602fd9-b1c1-4ae6-80dc-5cbafd6a3158'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-85ea34988ae059b7', 'type': 'function', 'function': {'name': 'search_web', 'arguments': '{"query": "langchain-mcp-adapters library"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'chatcmpl-tool-85ea34988ae059b7', 'type': 'function', 'function': {'name': 'search_web', 'arguments': '{"query": "langchain-mcp-adapters library"}'}}], 'reasoning': None, 'reasoning_content': None, 'token_usage': {'prompt_tokens': 309, 'total_tokens': 338, 'completion_tokens': 29, 'prompt_tokens_details': None}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.3-70b-instruct'}, id='lc_run--019c8bf2-281e-7a91-9d6d-e926cf2c83e3-

## Online MCP

In [17]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [ ]:
import re
from datetime import datetime
from typing import Any, Optional

from langchain_mcp_adapters.interceptors import MCPToolCallRequest


def _normalize_date_to_ddmmyyyy(value: Any) -> Optional[str]:
    if value is None:
        return None
    if not isinstance(value, str):
        return None
    s = value.strip()
    if not s:
        return None
    # Already in dd/mm/yyyy
    if re.fullmatch(r"\d{2}/\d{2}/\d{4}", s):
        return s
    # Try ISO formats
    for fmt in ("%Y-%m-%d", "%Y/%m/%d"):
        try:
            return datetime.strptime(s, fmt).strftime("%d/%m/%Y")
        except ValueError:
            pass
    # Remove ordinal suffixes: 1st, 2nd, 3rd, 4th, ...
    s2 = re.sub(r"(\d)(st|nd|rd|th)", r"\1", s, flags=re.IGNORECASE)
    # Common long-form formats
    for fmt in ("%B %d, %Y", "%b %d, %Y", "%B %d %Y", "%b %d %Y"):
        try:
            return datetime.strptime(s2, fmt).strftime("%d/%m/%Y")
        except ValueError:
            pass
    return None


def _coerce_int(value: Any) -> Optional[int]:
    if value is None:
        return None
    if isinstance(value, bool):
        return int(value)
    if isinstance(value, (int,)):
        return value
    if isinstance(value, float):
        return int(value)
    if isinstance(value, str):
        s = value.strip()
        if not s:
            return None
        try:
            # allow "0", "2", "2.0"
            return int(float(s))
        except ValueError:
            return None
    return None


async def sanitize_kiwi_flight_tool_args(request: MCPToolCallRequest, handler):
    # Only touch the flight search tool; let others pass through unchanged.
    if request.name != "search-flight":
        return await handler(request)

    args = dict(request.args or {})

    # Coerce flex ranges into numbers (the MCP tool schema expects numbers).
    for key in ("departureDateFlexRange", "returnDateFlexRange"):
        if key in args:
            coerced = _coerce_int(args.get(key))
            if coerced is None:
                # If blank/invalid, default to 0 rather than sending a string.
                args[key] = 0
            else:
                args[key] = coerced

    # Normalize dates to dd/mm/yyyy; omit returnDate if empty/invalid (one-way queries).
    if "departureDate" in args:
        normalized = _normalize_date_to_ddmmyyyy(args.get("departureDate"))
        if normalized is not None:
            args["departureDate"] = normalized

    if "returnDate" in args:
        normalized = _normalize_date_to_ddmmyyyy(args.get("returnDate"))
        if normalized is None:
            # Don’t send bad/empty returnDate; Kiwi MCP rejects it.
            args.pop("returnDate", None)
            args.pop("returnDateFlexRange", None)
        else:
            args["returnDate"] = normalized

    return await handler(request.override(args=args))

In [59]:
client = MultiServerMCPClient(
    {
        "kiwi-com-flight-search": {
            "transport": "streamable_http",
            "url": "https://mcp.kiwi.com",
        }
    },
    tool_interceptors=[sanitize_kiwi_flight_tool_args],
)

tools = await client.get_tools()

In [60]:
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model=model,
    tools=tools,
    #checkpointer=InMemorySaver(),
    system_prompt=
    "You are a helpful assistant that can search for flights. "
    "When calling tools: use dates in dd/mm/yyyy; use numbers (not strings) for flex ranges; "
    "and omit returnDate/returnDateFlexRange for one-way searches.",
)

In [61]:
question = HumanMessage(content="get me a direct flight from Toronto to New York on March 31st, 2026")

config  = {"configurable": {"thread_id": "2"}}
response = await agent.ainvoke(
    {"messages": [question]}, 
    #config=config
)

pprint(response)

{'messages': [HumanMessage(content='get me a direct flight from Toronto to New York on March 31st, 2026', additional_kwargs={}, response_metadata={}, id='fbc1c364-3003-4702-a348-e0471e09e4ab'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-b0db1995d2087ed9', 'type': 'function', 'function': {'name': 'search-flight', 'arguments': '{"flyFrom": "Toronto", "flyTo": "New York", "departureDate": "31/03/2026", "departureDateFlexRange": "0", "passengers": {"adults": 1, "children": 0, "infants": 0}, "cabinClass": "M", "sort": "price", "curr": "EUR", "locale": "en"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'chatcmpl-tool-b0db1995d2087ed9', 'type': 'function', 'function': {'name': 'search-flight', 'arguments': '{"flyFrom": "Toronto", "flyTo": "New York", "departureDate": "31/03/2026", "departureDateFlexRange": "0", "passengers": {"adu

In [62]:
print(response['messages'][-1].content)

Based on your request, I've found a direct flight from Toronto to New York on March 31st, 2026. The flight departs from Toronto Pearson International Airport (YYZ) and arrives at LaGuardia Airport (LGA). The departure time is 17:15 local time, and the arrival time is 18:55 local time. The total duration of the flight is 6000 seconds (or 1 hour and 40 minutes), and the price is 90 EUR. You can book this flight through the deep link: https://on.kiwi.com/GEqf5k. 

Please note that there are other flights available on the same day with different departure and arrival times, and prices. You can find more information about these flights in the output. 

Also, keep in mind that the prices and availability of flights may change over time, so it's always a good idea to check the latest information before booking your flight. 

I hope this helps, and I wish you a nice trip to New York! Did you know that New York City is home to over 20,000 restaurants, representing many different cultures and cu